In [ ]:
import pandas as pd
from numpy import loadtxt

# Cargar el dataset como matriz NumPy
dataset = loadtxt('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv', delimiter=',')

# Convertir a DataFrame de pandas
column_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
                'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']

data = pd.DataFrame(dataset, columns=column_names)

# Mostrar las primeras filas
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
import matplotlib.pyplot as plt

# Visualizar histogramas en 5 columnas
plt.figure(figsize=(20, 10))  # Ajustar el tamaño del gráfico

for i, col in enumerate(data.columns):
    plt.subplot((len(data.columns) + 4) // 5, 5, i + 1)  # Filas calculadas automáticamente
    plt.hist(data[col], bins=20, color='skyblue', edgecolor='black')
    plt.title(col)
    plt.tight_layout()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Visualizar boxplots en 5 columnas
plt.figure(figsize=(20, 10))  # Tamaño del gráfico

for i, col in enumerate(data.columns):
    plt.subplot((len(data.columns) + 4) // 5, 5, i + 1)  # 5 columnas
    plt.boxplot(data[col])
    plt.title(col)
    plt.tight_layout()

plt.show()


In [ ]:
data['Outcome'].value_counts()

In [ ]:
data.shape

## Red Neuronal Keras

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam

# Separar X e y
X = data.drop('Outcome', axis=1) # variables independientes o las que utilizare para predecir
y = data['Outcome'] # variable dependiente o la que quiero predecir

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

# Red neuronal más compacta
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_norm.shape[1],)), # capa de entradacon 8 columnas
    Dense(64, activation='relu'), # capa oculta o la hidden layer 
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid') # clasificacion binaria 0 o 1
])

# Compilar con SGD + momentum
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Evaluar
loss, accuracy = model.evaluate(X_test_norm, y_test, verbose=0)
print(f"Accuracy: {accuracy * 100:.2f}%")

import matplotlib.pyplot as plt

# Entrenar y guardar el historial
historial = model.fit(X_train_norm, y_train, epochs=100, batch_size=64, verbose=1, validation_data=(X_test_norm, y_test))



In [ ]:
import matplotlib.pyplot as plt

# Graficar pérdida
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(historial.history['loss'], label='Pérdida de entrenamiento')
plt.plot(historial.history['val_loss'], label='Pérdida de validación')
plt.title('Curva de pérdida')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.grid(True)

# Graficar accuracy
plt.subplot(1, 2, 2)
plt.plot(historial.history['accuracy'], label='Precisión de entrenamiento')
plt.plot(historial.history['val_accuracy'], label='Precisión de validación')
plt.title('Curva de precisión')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Obtener las predicciones del modelo como probabilidades
y_pred = model.predict(X_test_norm)

# Convertir probabilidades a clases binarias (0 o 1)
y_pred_classes = (y_pred > 0.5).astype(int).flatten()

# Generar la matriz de confusión
cm = confusion_matrix(y_test, y_pred_classes)

# Visualizar la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusión")
plt.xlabel('Predicciones')
plt.ylabel('Valores Reales')
plt.show()


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

# Obtener las predicciones del modelo como probabilidades
y_pred = model.predict(X_test_norm)

# Convertir probabilidades a clases binarias (0 o 1)
y_pred_classes = (y_pred > 0.5).astype(int).flatten()

# Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred_classes)

# Calcular especificidad
tn, fp, fn, tp = cm.ravel()  # Extraer valores de la matriz de confusión
specificity = tn / (tn + fp)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)  # Sensibilidad
f1 = f1_score(y_test, y_pred_classes)

# Imprimir resultados
print(f"Accuracy: {accuracy * 100:.1f}%")
print(f"Precisión: {precision * 100:.1f}%")
print(f"Sensibilidad (Recall): {recall * 100:.1f}%")
print(f"Especificidad: {specificity * 100:.1f}%")
print(f"Medida F1: {f1 * 100:.1f}%")


In [ ]:
# Guardar el modelo
model.save("modelo_diabetes.h5")

# Guardar el scaler con joblib o pickle
import joblib
joblib.dump(scaler, "scaler_diabetes.pkl")


In [ ]:
from tensorflow.keras.models import load_model
import joblib
import numpy as np

# Cargar modelo y scaler
modelo = load_model("modelo_diabetes.h5")
scaler = joblib.load("scaler_diabetes.pkl")


In [ ]:
# Variables en orden
columnas = [
    "Pregnancies", "Glucose", "BloodPressure", "SkinThickness",
    "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"
]

# Leer los valores como float desde el teclado
valores = []
print("Ingresa los valores del paciente:")
for col in columnas:
    val = float(input(f"{col}: "))
    valores.append(val)

# Convertir a array y escalar
valores_array = np.array(valores).reshape(1, -1)
valores_norm = scaler.transform(valores_array)

# Predecir
pred = modelo.predict(valores_norm)
clase = 1 if pred[0][0] >= 0.5 else 0

# Mostrar resultado
print(f"\nProbabilidad de diabetes: {pred[0][0]:.4f}")
print("Resultado:", "Positivo" if clase == 1 else "Negativo")
